In [2]:
import numpy as np
from psqt import *

In [301]:
components = np.load('results/trial11/c12.npy')
weights = np.load('results/trial11/w12.npy')

In [302]:
mean_raw_values = np.round(np.mean(raw_pesto_psqt, axis=0))
pesto_piece_values_matrix = np.tile(pesto_piece_values + mean_raw_values, 64).reshape(64, 12)
pesto_piece_values_matrix = np.round(pesto_piece_values_matrix / 9) * 9
# new_raw_values = pesto_psqt - pesto_piece_values_matrix

psqt = components @ np.concatenate([weights]) + pesto_piece_values_matrix

In [ ]:
np.mean((np.abs(psqt - pesto_psqt) / pesto_psqt) * importance_mask)

In [304]:
def psqt_to_cs_array(psqt):
    psqt = psqt.copy().astype(np.int32)
    psqt = psqt[:,np.array([0,2,4,6,8,10,1,3,5,7,9,11])]  # first mg and then eg
    output = "new int[][] {\n"
    for square in range(64):
        output += '\tnew int[] {'
        for piece in range(12):
            output += f'{psqt[square, piece]}' + (', ' if piece < 11 else '},\n')
    output += '};'
    return output

In [256]:
with open('cs_array.txt', 'w') as f:
    f.writelines(psqt_to_cs_array(psqt))

In [341]:
components = np.load('results/trial11/c10.npy')
weights = np.load('results/trial11/w10.npy')

In [342]:
mean_raw_values = np.round(np.mean(raw_pesto_psqt, axis=0))
pesto_piece_values_matrix = np.tile(pesto_piece_values + mean_raw_values, 64).reshape(64, 12)
pesto_piece_values_matrix = np.round(pesto_piece_values_matrix / 9) * 9 

In [343]:
def encode_component(component : np.array):
    component = component.astype(np.int32)
    ulong = 0x1000000000000000
    for bit in component[::-1]:
        ulong = (ulong << 1) | bit
    return ulong.astype(np.uint64) & 0xffffffffffffffff 


def encode_weights(weights : np.array):
    weights = weights.astype(np.uint8)
    hex_string = "0x"
    for weight in weights:
        # print("{0:#0{1}x}".format(weight,4))
        hex_string += "{0:#0{1}x}".format(weight,4)[-2:]
    return int(hex_string, 16)

def get_component_array(components : np.array):
    output = "new ulong[] {\n\t"
    for component in components.T:
        output += hex(encode_component(component)) + ',\n\t'
    output += '};'
    return output

def get_weight_array(all_weights: np.array, piece_values: np.array):
    output = "new decimal[] {\n\t"
    for weights in all_weights[:, [11, 9, 7, 5, 3, 1, 10, 8, 6, 4, 2, 0]]:
        output += str(encode_weights(weights)) + 'm,\n\t'
    output += str(encode_weights(np.round(piece_values[[11, 9, 7, 5, 3, 1, 10, 8, 6, 4, 2, 0]] / 9))) + 'm,\n\t'
    output += '};'
    return output
    

In [ ]:
print(get_component_array(components))

In [ ]:
print(get_weight_array(weights, pesto_piece_values_matrix[0,:]))